<div style="background-color: teal; padding: 10px;">
    <h3 style="color: white;">Experiment Two - Win/Loss Deployment</h3>
</div>

This is the **start** of deployment for a win/loss service.  The difference between this and the previous experiment is that we are building the functionality to predict the outcome of a game between two teams that have not played each other before.

This is a work in progress

<br>

#### Goal

To tackle the first part of a deployment of the experiment 2 win/loss model.

We'll just work on the mechanics of getting a request for two teams, stitching the team's stats together, and returning a prediction.

<br>

#### Background

In real-life we woud be querying a match between two teams.

We would not have a record specifically for those teams facing off against each other, so we would have to create one.

So we take the offense and defense stats of each team and stitch them together into a 'new' X_Test record.

We have already used ll of the 2022 season to validate the train the model, but we can still re-use it to buid the deployment.  The results are not as critical as the basic functionality.

So we'll:

- feed the 2022 season one at a time - just providing the two team names


- the deployment will already have stats for each team from the ETL we've already created,


- so we'll stitch them together into a single record


#### Scope

- We'll just work on the mechanics of stitching two teams's stats together into a proper input record and submitting it to the model, and validating what we already know - who really won or lost

- This is still a 'toy' model.  We are staring on the deployment as a learning process.

- A major todo is that we need to be able to handle the cases in early weeks when our data might not be as good

- For this run we'll just get the mechanics of stitching the teams together into a proper input record and submitting it to the model, and validating what we already know - who really won or lost


#### Input
We can't use the same data we used to train or test the model, because those game were already played and we are trying to build the mechanics to predict a game that has not yet occurred.  So we start with the data we used to build the train set.

- tmp_defense_week_features.parquet
- tmp_offense_week_features.parquet
- pbp_actions.parquet

- team_week_model.keras
- team_week_scaler.pkl
-

#### Steps:

- use the 2022 to feed in game requests one-at-at-time to the pre-trained model
- predict each game and compare to the actual result


#### Output

- personal satisfaction

#### Next Steps

- hide any future data from the model, providing only previous weeks, and re-tune the model

- handle the cases in early weeks when our data might not be as good

- feed the 2022 season one at a time - only providing weeks less than the week we are requesting

- complete the ETL to pick up data weekly from the NFL site - so we always have the latest data

- depending on timeliness it could be necessary to scrape the data from the source raher than relying on nflverse



<div style="background-color: teal; padding: 10px;">
    <h3 style="color: white;">imports</h3>
</div>

We'll take care of some housekeeping here - importing libraries, setting up the environment, etc.

In [9]:
import os
import sys

sys.path.append(os.path.abspath("../src"))

In [10]:


from src.nfl_04_merge_game_features import load_file
import warnings

warnings.filterwarnings('ignore')


In [11]:
from src import *

<div style="background-color: teal; padding: 10px;">
    <h3 style="color: white;">Feed the 2022 games</h3>
</div>

- feed the 2022 season one at a time - just providing the two team names, then

- the deployment will already have stats for each team from the ETL we've already created,
    so we'll stitch them together into a single record



#### helper functions to feed the data in

In [12]:
def request_generator(season, week=5):
    pbp_actions_df = load_file(get_config('data_directory'), get_config('action_week_prep'))
    pbp_actions_df = pbp_actions_df.loc[(pbp_actions_df['season'] == season) & (pbp_actions_df['week'] == week)] \
        .sort_values(by=['week'])[['season', 'week', 'game_id', 'home_team', 'away_team', 'home_final_score', 'away_final_score']] \
        .drop_duplicates()

    print(f"Will generate {pbp_actions_df.shape[0]} requests")

    for index, row in pbp_actions_df.iterrows():
        game = row[['season', 'week', 'game_id', 'home_team', 'away_team', 'home_final_score', 'away_final_score']].to_dict()
        game['win_lose'] = 1 if game['home_final_score'] >= game['away_final_score'] else 0
        yield game


#### instantiate the deployment class
This class loads the model, the scaler, and the stats for each team
It handles requests for creating a 'match' between two teams
It performs the stitching of the two teams into a single record

In [13]:
from src.serve.serve_model import NFLWinLossDeployment

deployment = NFLWinLossDeployment(season=2022, week=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 246)               8610      
                                                                 
 dense_1 (Dense)             (None, 164)               40508     
                                                                 
 dense_2 (Dense)             (None, 100)               16500     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 64)                6464      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 3

#### create the request generator for week 1 of 2022

In [14]:
requests = request_generator(season=2022, week=1)

#### feed requests into the deployment one at a time

In [15]:

# Iterate through the 2022 request generator and print each row
stats = []
total = 0
corrects = 0
for row in requests:
    total += 1

    # save the actual result
    y_actual = row['win_lose']

    # create a match between the two teams - stitching thier stats into one input record
    match_df = deployment.create_match(row['home_team'], row['away_team'])

    # perform the prediction
    y_pred = deployment.predict(match_df[configs.ml_win_lose_features])

    # keep track of how we did
    stats.append(dict(home=row['home_team'], away=row['away_team'], y_actual=y_actual, y_pred=y_pred))
    if y_actual == y_pred:
        corrects += 1
    print(f"Request a prediction for {row['home_team']} vs {row['away_team']}\t\tresults: prediction: {y_pred}, actual: {y_actual}  \t{'correct' if y_actual == y_pred else ''} ")
    if total > 20:
        break



2023-07-27 12:09:49,947 - INFO - Reading from tmp_weekly_play_actions


Will generate 16 requests
Request a prediction for ARI vs KC		results: prediction: 0, actual: 0  	correct 
Request a prediction for SEA vs DEN		results: prediction: 1, actual: 1  	correct 
Request a prediction for DAL vs TB		results: prediction: 0, actual: 0  	correct 
Request a prediction for DET vs PHI		results: prediction: 0, actual: 0  	correct 
Request a prediction for HOU vs IND		results: prediction: 0, actual: 1  	 


2023-07-27 12:09:50.175038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Request a prediction for LA vs BUF		results: prediction: 0, actual: 0  	correct 
Request a prediction for CAR vs CLE		results: prediction: 0, actual: 0  	correct 
Request a prediction for CHI vs SF		results: prediction: 1, actual: 1  	correct 
Request a prediction for MIA vs NE		results: prediction: 1, actual: 1  	correct 
Request a prediction for CIN vs PIT		results: prediction: 0, actual: 0  	correct 
Request a prediction for NYJ vs BAL		results: prediction: 0, actual: 0  	correct 
Request a prediction for WAS vs JAX		results: prediction: 1, actual: 1  	correct 
Request a prediction for MIN vs GB		results: prediction: 1, actual: 1  	correct 
Request a prediction for TEN vs NYG		results: prediction: 0, actual: 0  	correct 
Request a prediction for ATL vs NO		results: prediction: 1, actual: 0  	 
Request a prediction for LAC vs LV		results: prediction: 1, actual: 1  	correct 


#### print the results

In [16]:
print(f"total requests: {total},  correct predictions: {corrects},  accuracy: {corrects / total}")


total requests: 16,  correct predictions: 14,  accuracy: 0.875
